In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from qiskit import QuantumCircuit
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZFeatureMap
from qiskit.quantum_info import SparsePauliOp
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.neural_networks import EstimatorQNN
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

algorithm_globals.random_seed = 12345

def conv_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)
    target.cx(1, 0)
    target.rz(np.pi / 2, 0)
    return target

def conv_layer(num_qubits, param_prefix):
    qc = QuantumCircuit(num_qubits, name="Convolutional Layer")
    qubits = list(range(num_qubits))
    param_index = 0
    params = ParameterVector(param_prefix, length=num_qubits * 3)
    for q1, q2 in zip(qubits[0::2], qubits[1::2]):
        qc = qc.compose(conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
        qc.barrier()
        param_index += 3
    for q1, q2 in zip(qubits[1::2], qubits[2::2] + [0]):
        qc = qc.compose(conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()

    qc = QuantumCircuit(num_qubits)
    qc.append(qc_inst, qubits)
    return qc

def pool_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)

    return target

def pool_layer(sources, sinks, param_prefix):
    num_qubits = len(sources) + len(sinks)
    qc = QuantumCircuit(num_qubits, name="Pooling Layer")
    param_index = 0
    params = ParameterVector(param_prefix, length=num_qubits // 2 * 3)
    for source, sink in zip(sources, sinks):
        qc = qc.compose(pool_circuit(params[param_index : (param_index + 3)]), [source, sink])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()

    qc = QuantumCircuit(num_qubits)
    qc.append(qc_inst, range(num_qubits))
    return qc

/var/folders/mh/v3x6dz8534gcdj91_hh0bdph0000gn/T/ipykernel_22608/4190158425.py:6: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms.optimizers import COBYLA


In [2]:
# Define paths to your train data folders
train_data_dir = 'test'

# Set image dimensions and batch size
img_width, img_height = 6,6
batch_size = 32

In [3]:
# Create an ImageDataGenerator for data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generate training data
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    color_mode='rgb'
)

# Extract X_train (input images) and Y_train (labels)
X_train = []
Y_train = []

for images, labels in train_generator:
    X_train.extend(images)
    Y_train.extend(labels)
    
    if len(X_train) >= len(train_generator.filenames):
        break

X_train = np.array(X_train)
Y_train = np.array(Y_train)

# Print class labels and their corresponding indices
class_indices = train_generator.class_indices
print("Class Indices:", class_indices)

# Convert labels to one-hot encoding
num_classes = len(class_indices)
Y_train_one_hot = tf.keras.utils.to_categorical(Y_train, num_classes=num_classes)

print("X_train shape:", X_train.shape)
print("Y_train_one_hot shape:", Y_train_one_hot.shape)

'''
Output:
Found 7178 images belonging to 7 classes.
Class Indices: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
X_train shape: (7178, 48, 48, 3)
Y_train_one_hot shape: (7178, 7, 7)
'''

Found 7178 images belonging to 7 classes.
Class Indices: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
X_train shape: (7178, 6, 6, 3)
Y_train_one_hot shape: (7178, 7, 7)


"\nOutput:\nFound 7178 images belonging to 7 classes.\nClass Indices: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}\nX_train shape: (7178, 48, 48, 3)\nY_train_one_hot shape: (7178, 7, 7)\n"

In [4]:
from sklearn.decomposition import PCA

# Confirming the images are grayscale by checking if all channels have the same values
assert np.all(X_train[:,:,:,0] == X_train[:,:,:,1]) and np.all(X_train[:,:,:,0] == X_train[:,:,:,2])

# Dropping the redundant channels
X_train = X_train[:,:,:,0]

# Flattening the images
X_train_flat = X_train.reshape(X_train.shape[0], -1)

# Applying PCA
'''pca = PCA(n_components=6)
X_train_pca = pca.fit_transform(X_train_flat)

X_train_pca.shape'''

'pca = PCA(n_components=6)\nX_train_pca = pca.fit_transform(X_train_flat)\n\nX_train_pca.shape'

In [5]:

# Assuming the necessary libraries and functions have been imported previously
# Extracting the shape of X_train and Y_train_one_hot
X_train_shape = X_train_flat.shape
Y_train_one_hot_shape = Y_train_one_hot.shape

X_train_shape, Y_train_one_hot_shape


((7178, 36), (7178, 7, 7))

In [6]:
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

In [14]:
# Adjust the feature map for 6 qubits
feature_map = ZFeatureMap(6)

ansatz = QuantumCircuit(6, name="Ansatz")

# First Convolutional Layer
ansatz.compose(conv_layer(6, "с1"), list(range(6)), inplace=True)

# First Pooling Layer
ansatz.compose(pool_layer([0, 1, 2], [3, 4, 5], "p1"), list(range(6)), inplace=True)

# Second Convolutional Layer
# Note: We've reduced the convolutional layer to cover qubits 3 to 5
ansatz.compose(conv_layer(3, "c2"), list(range(3, 6)), inplace=True)

# Second Pooling Layer
# Note: We've adjusted the pooling layer to cover qubits 3 to 4
ansatz.compose(pool_layer([0, 1], [2], "p2"), list(range(3, 6)), inplace=True)

# Third Convolutional Layer
# Note: This convolutional layer now only covers qubit 5
ansatz.compose(conv_layer(1, "c3"), list(range(5, 6)), inplace=True)

# We will omit the third pooling layer as it doesn't make sense with just 1 qubit

# Combining the feature map and ansatz
circuit = QuantumCircuit(7)  # Adjusted for 6 qubits + 1 ancilla
circuit.compose(feature_map, range(6), inplace=True)
circuit.compose(ansatz, range(6), inplace=True)

observable = [SparsePauliOp.from_list([('Z' + 'I' * i + 'Z' + 'I' * (5 - i), 1)]) for i in range(6)]

# we decompose the circuit for the QNN to avoid additional data copying
qnn = EstimatorQNN(
    circuit=circuit.decompose(),
    observables=observable,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
)

from qiskit.algorithms.optimizers import ADAM
optimizer = ADAM(maxiter=100)

classifier = NeuralNetworkClassifier(
    qnn,
    optimizer=optimizer,
    callback=callback_graph
)

In [15]:
print(feature_map.parameters)

ParameterView([ParameterVectorElement(x[0]), ParameterVectorElement(x[1]), ParameterVectorElement(x[2]), ParameterVectorElement(x[3]), ParameterVectorElement(x[4]), ParameterVectorElement(x[5])])


In [17]:
objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)
classifier.fit(X_train, Y_train_one_hot)

'''output

capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.
'''

# score classifier
#print(f"Accuracy from the train data : {np.round(100 * classifier.score(x, y), 2)}%")

KeyboardInterrupt: 

: 